### data creation

In [2]:
def cut_data_by_qtime(qtime=0.5):
    '''le problème principal est que la liste d articles étaient presque complètes avant la première interaction
        en cutant par un quantile du click_timestamp, on diminuer le nombre de clics mais pas le nombre d'articles
        et si on cut par un quantile items.created_at_ts, on aura plus d interactions
        
        dans l ideal il faut cutter les items par un quantile items.created_at_ts
                                et les interactions par un quantile click_timestamp
                                en virant toutes les interactions avec des items cuttés
                                mais on peut renconter un grand saut dans le nombre d interactions'''

    cut_click=clics.click_timestamp.quantile(qtime)
    #cut_items=items.created_at_ts.quantile(q)
    #fusion items et items2vec "embedding"
    items_=pd.concat([items,pd.DataFrame(items2vec)], axis=1)
    #data cut
    clics__=clics.loc[clics.click_timestamp<cut_click].copy()
    items__=items_.loc[items_.created_at_ts<cut_click].copy()
    
    #merge clics & items, pour retirer les articles non connus
    data=clics__.loc[clics__.click_article_id.isin(set(items__.article_id))].copy()
    data.rename(columns={'click_article_id':'article_id'}, inplace = True)
    #display(data)
    #1 ligne par item interagi pour chaque user
    data_map=data[['user_id','article_id']].groupby(['user_id', 'article_id']).size().to_frame().reset_index()
    data_map.columns=['user_id','article_id','nb']
    #display(data_map)
    #extraction item2vec
    items2vec_=items__.iloc[:,5:].to_numpy()
    #creation des dicos inner_id/raw_id et raw_id/inner_id
    dico_inner_raw_iid = dict( (inner_iid,iid) for inner_iid,iid in enumerate(items__.article_id ))
    dico_raw_inner_iid = dict( (iid,inner_iid) for inner_iid,iid in enumerate(items__.article_id ))
    return data_map, items__, items2vec_, dico_inner_raw_iid, dico_raw_inner_iid

In [3]:
#globo user data
import pandas as pd
#article content embedding : np matrix 364047 articles / 250D
#embeddings can be used by Neural Networks to represent their content
items2vec = pd.read_pickle(r'data/news-portal-user-interactions-by-globocom/articles_embeddings.pickle')
#metadata info about 364047 articles
items = pd.read_csv('data/news-portal-user-interactions-by-globocom/articles_metadata.csv', delimiter = ',')
#all_click
for i in range(365):
    temp=pd.read_csv('data/news-portal-user-interactions-by-globocom/clicks/clicks/clicks_hour_'+"%03d" % i+'.csv', delimiter = ',')
    if i==0:
        clics=temp
    else:
        clics=pd.concat([clics, temp])
clics=clics.reset_index(drop=True)

data_map,items_df, i2vec, dic_ir, dic_ri=cut_data_by_qtime(qtime=0.01)

### serialization

In [5]:
### dump 
import os
import pickle
local_path = "./2AzBlob_2"
#os.mkdir(local_path)

#on binarise/serialise le numpy array du word embedding
local_file='data_map.pkl'
file_path = os.path.join(local_path, local_file)
with open(file_path, 'wb') as f:
    pickle.dump(data_map, f)
    
local_file='items_df.pkl'
file_path = os.path.join(local_path, local_file)
with open(file_path, 'wb') as f:
    pickle.dump(items_df, f)
    
local_file='i2vec.pkl'
file_path = os.path.join(local_path, local_file)
with open(file_path, 'wb') as f:
    pickle.dump(i2vec, f)

local_file='dic_ir.pkl'
file_path = os.path.join(local_path, local_file)
with open(file_path, 'wb') as f:
    pickle.dump(dic_ir, f)

local_file='dic_ri.pkl'
file_path = os.path.join(local_path, local_file)
with open(file_path, 'wb') as f:
    pickle.dump(dic_ri, f)

### to blob storage

In [7]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

try:
    print("Azure Blob Storage v" + __version__ + " - Python quickstart sample")
        # recuperation de la chaine de connexion au azure blob storage à partir de la variabale environmentale
    connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
        # Creation du service_client blob qui permettra de creer un conteneur
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
        # Creation d'un nom unique aleatoire grace a la fonction uuid4 pour le conteneur blob
    container_name = str(uuid.uuid4())
        # Creation du conteneur
    container_client = blob_service_client.create_container(container_name)


    
    toblob_list=['data_map.pkl','items_df.pkl','i2vec.pkl','dic_ir.pkl','dic_ri.pkl']

    # Create a blob client using the local file name as the name for the blob
    for i in toblob_list:
        local_file = i
        upload_file_path = os.path.join(local_path, local_file)
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=local_file)

        print("\nUploading to Azure Storage as blob:\n\t" + local_file)

        # Upload the created file
        with open(upload_file_path, "rb") as data:
            blob_client.upload_blob(data)


except Exception as ex:
    print('Exception:')
    print(ex)

Azure Blob Storage v12.12.0 - Python quickstart sample

Uploading to Azure Storage as blob:
	data_map.pkl

Uploading to Azure Storage as blob:
	items_df.pkl

Uploading to Azure Storage as blob:
	i2vec.pkl

Uploading to Azure Storage as blob:
	dic_ir.pkl

Uploading to Azure Storage as blob:
	dic_ri.pkl


In [8]:
print("\nListing blobs...")

# List the blobs in the container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print("\t" + blob.name)


Listing blobs...
	data_map.pkl
	dic_ir.pkl
	dic_ri.pkl
	i2vec.pkl
	items_df.pkl


### recup des données du azure blob storage
    * attention pas les memes fonctions pour les bindings azure functions

In [9]:
%%time
#chargement du blob
#blob_client = BlobClient.from_connection_string(connection_string, container_name, blob_name)
blob_client = blob_service_client.get_blob_client(container=container_name, blob='data_map.pkl')
data_map_=pickle.loads(blob_client.download_blob(0).readall())
#blob_client = blob_service_client.get_blob_client(container=container_name, blob='items_df.pkl')
#items_df_=pickle.loads(blob_client.download_blob(0).readall())
blob_client = blob_service_client.get_blob_client(container=container_name, blob='i2vec.pkl')
i2vec_=pickle.loads(blob_client.download_blob(0).readall())
blob_client = blob_service_client.get_blob_client(container=container_name, blob='dic_ir.pkl')
dic_ir_=pickle.loads(blob_client.download_blob(0).readall())
blob_client = blob_service_client.get_blob_client(container=container_name, blob='dic_ri.pkl')
dic_ri_=pickle.loads(blob_client.download_blob(0).readall())

CPU times: total: 2.75 s
Wall time: 11.6 s


In [10]:
container_name

'5130f488-4775-417a-b715-1eeafb91e572'

### content based classe

In [11]:
import numpy as np
from sklearn import preprocessing
import scipy
from sklearn.metrics.pairwise import cosine_similarity

class ContentBasedRecommender:
    '''le fit calcule les users profiles'''
    
    MODEL_NAME = 'Content-Based'
    

    def __init__(self,data_map,i2vec,dic_ri,dic_ir):
        '''constructeur'''
        #dico inner raw iid
        self.dic_ir = dic_ir
        #dico raw inner iid
        self.dic_ri = dic_ri
        self.items_embedding = i2vec
        #les train interactions servent à:
            #calculer les profils
            #ignorer les interactions dans la reco
        self.train_user_interact = data_map
        
    ###############################################################
    #profilage des users avec le fit
        
    def _build_users_profile(self,uid, click_df,emb_matrix,dic_ri):
        '''calcul du profil embedding pour un uid
        à partir des interactions du train'''
        #on recupere le dataframe d'interaction pour l'uid ciblé
        click_uid_df = click_df.loc[click_df.user_id==uid]
        #on recupere les embeddings des items vus par l'uid
        user_item_profiles = np.array([emb_matrix[dic_ri[iid]] for iid in click_uid_df['article_id']])
        #on recupere le nombre de click sur les articles par l'uid afin d apporter une ponderation à la moyenne des embedding pour le calcul du profil
        user_item_strengths = np.array(click_uid_df['nb']).reshape(-1,1) #-1 veut dire unknow dim
        #on pondere la localisation embedding de chaque item par le nombre d'interactions puis on somme afin d'obtenir le profile qui est un barycentre
        user_item_strengths_weighted_avg = np.sum(np.multiply(user_item_profiles,user_item_strengths), axis=0) / np.sum(user_item_strengths)
        user_profile_norm = preprocessing.normalize(np.reshape(user_item_strengths_weighted_avg,(1,250)))
        return user_profile_norm

    def _build_users_profiles(self,click_df, emb_matrix,dic_ri): 
        '''calcul des profils de tous les uid sous forme de dic {uid:profil}
        à partir des interactions du train'''
        user_profiles = {}
        for uid in click_df.user_id.unique():
            user_profiles[uid] = self._build_users_profile(uid, click_df, emb_matrix, dic_ri)
        return user_profiles
    
    def fit(self):
        '''calcul des profils utilisateurs'''
        self.user_profiles=self._build_users_profiles(self.train_user_interact,self.items_embedding,self.dic_ri)
        #return self.user_profiles
        
    ###############################################################
    
    
    def get_model_name(self):
        '''renvoie le nom du modèle'''
        return self.MODEL_NAME
    

    def _get_similar_items_to_user_profile(self, uid):#, topn=100000):
        '''renvoie une liste d items similaires au profil de l'uid
            liste de couples (raw iid, cosine) triés par cosine decroissant'''
        #on calcul distances profil uid vs articles : sous forme 2D 1 ligne / nombre iid colonnes dans l'ordre inner du wemb
        cosine_similarities = cosine_similarity(self.user_profiles[uid], self.items_embedding)
        #on tri par distance: argsort donne les indices par scores croissants /flatten pour passer en 1D
        #on recupère un array des inner_iid de taille topn : les plus proches du profil sont à droite
        similar_indices = cosine_similarities.argsort().flatten()#[-topn:]
                                                        ################################################################################
        #on tri : sorted(iterable, key=None, reverse=False)
        # l'iterable est une liste de couples (raw_iid, cosine similarites)
        #on tri par similarité décroissante
        similar_items = [(self.dic_ir[i], cosine_similarities[0,i]) for i in similar_indices[::-1]]
        return similar_items
    
    #methode qui recommande en ignorant les articles deja vus
    def recommend_items(self, uid, topn=5):
        '''renvoie une liste de reco similaires au profil de l uid
            triés par cosine decroissant
             sans les interactions du train
                sous forme de dataframe'''
        #on recupere la liste brute de 1200 reco classés par cosine decroissant
        similar_items = self._get_similar_items_to_user_profile(uid)
        #on recupere la liste des iid a ignorer car deja vus
        #items_to_ignore = self.train_user_interact.loc[self.train_user_interact.user_id==uid,"article_id"].tolist()
        items_to_ignore = set(self.train_user_interact.loc[self.train_user_interact.user_id==uid].article_id)
        #on enleve les deja vus
        #similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        recommendations_df = pd.DataFrame(similar_items, columns=['article_id', 'cb_cosine_with_profile'])
        reco = recommendations_df.loc[recommendations_df.article_id.isin(items_to_ignore)==False].copy()
        if topn>0:
            reco=reco.head(topn)
        return reco

### colab filtering classe

In [12]:
from surprise import Reader, Dataset,SVDpp
class CollabFiltRecommender:
    
    MODEL_NAME = 'Collaborative-Filtering-SVDpp'
    
    #constructeur
    def __init__(self,data_map):
        '''constructeur'''
        #les train interactions servent à:
            #calculer les profils
            #ignorer les interactions dans la reco
        self.train_user_interact = data_map
        
    #methode qui renvoie le nom du modele    
    def get_model_name(self):
        return self.MODEL_NAME
    
    #fit du modèle
    def fit(self):
        'fit du modele SVDpp'
        #constructeur du reader
        reader = Reader(rating_scale=(0,5))
        #lecture du train par surprise
        data = Dataset.load_from_df(self.train_user_interact[['user_id','article_id','nb']], reader)
        #costruction du trainset surprise 
        trainset = data.build_full_trainset()
        self.algo = SVDpp(n_epochs=20, lr_all=0.007, reg_all=0.1)
        self.algo.fit(trainset)

    
    def recommend_items(self,uid,topn=5):
        '''renvoie une liste de prediction pour un uid
            le SVD ne peut sortir des prédictions que pour les uid et iid dispo dans le train'''
        #prediction fonctionne avec les raws id
        #https://towardsdatascience.com/difference-between-apply-and-transform-in-pandas-242e5cf32705
        iid_to_ignore=set(self.train_user_interact.loc[self.train_user_interact.user_id==uid].article_id)
        items2pred=pd.DataFrame(set(self.train_user_interact.article_id)-iid_to_ignore,columns=['article_id'])
        items2pred['pred']=items2pred['article_id'].apply(lambda x:self.algo.predict(uid=uid, iid=x)[3])
        #pred['detail']=pred['article_id'].apply(lambda x:cf.predict(uid=uid, iid=x)[4])
        if topn==0:
            recommendations_df=items2pred.loc[:,['article_id','pred']].sort_values(by='pred', ascending=False)
        else:
            recommendations_df=items2pred.loc[:,['article_id','pred']].sort_values(by='pred', ascending=False).head(topn)

        return recommendations_df

### pop filtering classe

In [13]:
class PopularityFiltRecommender:
    
    MODEL_NAME = 'Popularity-Filtering'
    
    #constructeur
    def __init__(self,data_map):
        '''constructeur'''
        self.train_user_interact = data_map
        
    #methode qui renvoie le nom du modele    
    def get_model_name(self):
        return self.MODEL_NAME
    
    #fit du modèle
    def fit(self):
        self.raw_reco=self.train_user_interact[['nb','article_id']].groupby(['article_id']).sum().sort_values(by=['nb'],ascending=False).reset_index()

    def recommend_items(self,uid,topn=5):
        '''renvoie une liste de prediction pour un uid'''
        if topn==0:
            recommendations_df=self.raw_reco
        else:
            recommendations_df=self.raw_reco.head(topn)
        return recommendations_df

### hybrid filtering classe

In [14]:
class HybridRecommender:
    
    MODEL_NAME = 'Hybrid-Filtering'
    
    #constructeur
    def __init__(self,data_map,i2vec,dic_ri,dic_ir):
        '''constructeur'''
        self.train_user_interact = data_map
        self.dic_ir = dic_ir
        self.dic_ri = dic_ri
        self.items_embedding = i2vec
    
    def get_model_name(self):
        return self.MODEL_NAME
    
    #fit du modèle
    def fit(self):
        '''le fit consiste a fitter les sous modèles'''
        self.cf_model = CollabFiltRecommender(self.train_user_interact)
        self.cf_model.fit()
        self.cb_model = ContentBasedRecommender(self.train_user_interact,self.items_embedding,self.dic_ri,self.dic_ir)
        self.cb_model.fit()
        self.pf_model = PopularityFiltRecommender(self.train_user_interact)
        self.pf_model.fit()
        
    def recommend_items(self,uid,topn=5):
        '''renvoie une liste de prediction pour un uid'''
        #si uid pas connu du train: seul le popularity recommander fonctionne dans notre cas
        #on pourrait également concevoir un social reco basé sur un social profil, encore faudrait il avoir assez d 'infos'
        if uid not in set(self.train_user_interact.user_id):
            #parfait quand l'uid n'est pas connu du train, n a pas  d'interactions
            reco=self.pf_model.recommend_items(uid,0)
        else:
                # content based : permet de donner des iid qui n'ont pas d'interactions mais qui étaient disponibles
            reco_cb=self.cb_model.recommend_items(uid,0)
                # pour la normalisation, le but est de garder les meilleurs, donc les outliers positifs
                #il vaut mieux dans ce cas diviser par le max que par une standard deviation 
                #qui ne rendra pas forcement les outliers similaires entre facteurs pour une pondéaration
            #reco_cb['norm_cb']=(reco_cb.cb_cosine_with_profile-reco_cb.cb_cosine_with_profile.median())/(reco_cb.cb_cosine_with_profile.max()-reco_cb.cb_cosine_with_profile.median())
            reco_cb['rank_cb']=reco_cb.cb_cosine_with_profile.rank(ascending=False)
                # donne de bons resultats mais necessite des uid et des iid qui ont des interactions dans le train
            reco_cf=self.cf_model.recommend_items(uid,0)
            #reco_cf['norm_cf']=(reco_cf.pred-reco_cf.pred.median())/(reco_cf.pred.max()-reco_cf.pred.median())
            reco_cf['rank_cf']=reco_cf.pred.rank(ascending=False)
                #comment les pondérer? 80/20 normalisé par exemple?
                #en gros la pondération est anecdotique: on privilegie CF et quand on a pas de CF, on met la note de CB
            reco = reco_cb.merge(reco_cf,how='outer', on='article_id')
                #on fill les norm_cf vides (iid inconnu du train) par norm_cv avant pondération
                #on pourait également filler les iid peu interagis afin de favoriser le content dans ce cas la
            reco.loc[reco['rank_cf'].isnull(),'rank_cf'] = reco['rank_cb']
            reco['rank']=0.99999*reco['rank_cf']+0.00001*reco['rank_cb']
            
            reco=reco.sort_values(by='rank', ascending=True)


        if topn==0:
            recommendations_df=reco
        else:
            recommendations_df=reco.head(topn)
            
        return recommendations_df.reset_index(drop=True)
        #return recommendations_df

### calcul top 5 pour tous les uid

In [15]:
%%time
hr_model = HybridRecommender(data_map_,i2vec_,dic_ri_,dic_ir_)
hr_model.fit()

CPU times: total: 6.52 s
Wall time: 6.52 s


In [19]:
print('nb de users:',len(set(data_map_.user_id)))

nb de users: 10457


In [20]:
hr_model.recommend_items(5)

,article_id,cb_cosine_with_profile,rank_cb,pred,rank_cf,rank
0,274999,0.781554,1.0,NaN,1.0,1.00000
1,207742,0.600974,2984.0,1.138642,1.0,1.02983
2,203789,0.773637,2.0,NaN,2.0,2.00000
3,206967,0.773242,3.0,NaN,3.0,3.00000
4,277263,0.214929,178959.0,1.113861,2.0,3.78957


In [27]:
%%time
reco={}
#0 n'étant pas un user id valide il servira à stocker le popularity reco
reco[0]=hr_model.recommend_items(0).article_id.to_list()
for c,i in enumerate(set(data_map_.user_id)):
    if c%(int(len(set(data_map_.user_id))/100))==0:
        print('users processed:' ,(100*c/len(set(data_map_.user_id))),'%',end='\r')
    reco[i]=hr_model.recommend_items(i).article_id.to_list()

CPU times: total: 2h 26min 47s1019 %%
Wall time: 2h 13min 33s


In [43]:
#il faut obligatoiurement donner un id qui ne soi pas 0, en string ca passe
reco_items_to_create=[]
for key, value in reco.items():
    reco_items_to_create.append({'id':str(key),'userId':key,'reco':value})
    
local_file='reco_items_to_create.pkl'
file_path = os.path.join(local_file)
with open(file_path, 'wb') as f:
    pickle.dump(reco_items_to_create, f)

### connexion et creation du conteneur cosmodb

In [38]:
from azure.cosmos import exceptions, CosmosClient, PartitionKey

# Initialize the Cosmos client
endpoint = "https://nab-cosmodb-account.documents.azure.com:443/"
key = 'qMtk0HrT6bHZgHOeKyqAO8Q7LKlwgCdIn1oMIfpxkf9IqvJdT0ZGqfdzVdfRhBA2Sx7DgfMIoyrbqTlp87a4ew=='

# <create_cosmos_client>
client = CosmosClient(endpoint, key)
# Create a database
database_name = 'RecoP9Database'
database = client.create_database_if_not_exists(id=database_name)
# Create a container
# en general il faut des partitions keys qui suporteront equitablement les requetes pour optimiser la charge
# d'ou le choix d'un bon partition key
container_name = 'RecoP9Container'
container = database.create_container_if_not_exists(
    id=container_name, 
    partition_key=PartitionKey(path="/userId"),
    #offer_throughput=400 -> pas dispo en serverless
)

### chargement des 10457 items dans cosmo db: item par item, c est pas le plus efficace, il doit exister un autre moyen

In [44]:
%%time
# <create_item>
for reco_item in reco_items_to_create:
    container.create_item(body=reco_item)

CPU times: total: 1min 1s
Wall time: 29min 13s


In [51]:
#test cosmodb
query = "SELECT c.reco FROM c WHERE c.id in ('7','0')"

items = list(container.query_items(
    query=query,
    enable_cross_partition_query=True
))

request_charge = container.client_connection.last_response_headers['x-ms-request-charge']

print('Query returned {0} items. Operation consumed {1} request units'.format(len(items), request_charge))
print(items[0]['reco'])

Query returned 2 items. Operation consumed 2.95 request units
[157519, 277263, 159495, 156690, 118773]


In [61]:


try:
    print(items[2])
    pass
except IndexError:
    print(items[1])

{'reco': [285781, 337465, 97948, 316239, 232088]}


In [ ]:
"SELECT c.reco FROM c WHERE c.userId in (0,StringToNumber({userId}))",